In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [3]:
nc = pd.read_excel('./data/corr/nc_total.xlsx')
sdi = pd.read_excel('./data/corr/sdi_total.xlsx')
ssem = pd.read_excel('./data/corr/삼성전기_total.xlsx')
hynix = pd.read_excel('./data/corr/하이닉스_total.xlsx')

In [4]:
nc = nc.dropna(axis = 1)
sdi = sdi.dropna(axis = 1)
ssem = ssem.dropna(axis = 1)
hynix = hynix.dropna(axis = 1)

In [5]:
nc_corr = nc.corr(method = 'pearson')
df_nc = pd.DataFrame(nc_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_nc.columns = ['지표','종가']

sdi_corr = sdi.corr(method = 'pearson')
df_sdi = pd.DataFrame(sdi_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_sdi.columns = ['지표','종가']

ssem_corr = ssem.corr(method = 'pearson')
df_ssem = pd.DataFrame(ssem_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_ssem.columns = ['지표','종가']

hynix_corr = hynix.corr(method = 'pearson')
df_hynix = pd.DataFrame(hynix_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_hynix.columns = ['지표','종가']

In [6]:
list_corr = []
order = 15

for i in range(len(df_nc.index[:order])):
    list_corr.append(df_nc.index[:order][i])
    list_corr.append(df_sdi.index[:order][i])
    list_corr.append(df_ssem.index[:order][i])
    list_corr.append(df_hynix.index[:order][i])

variable = pd.Series(list_corr).value_counts()
variable = pd.DataFrame(variable).reset_index()
variable.columns = ['var_index', 'number']

In [8]:
var = []
for i in range(len(variable)):
    var.append(df_nc['지표'][variable.index[i]])
var = pd.DataFrame(var)
var.columns = ['var']

In [77]:
corr

,var_index,var,number
0,24,거래소코드,4
1,14,[공통]총자본증가율(IFRS연결),4
2,47,[공통]자기자본증가율(IFRS연결),4
3,46,[공통]매출액증가율(IFRS연결),4
4,45,[공통]정상영업이익증가율(IFRS연결),4
5,44,[공통]순이익증가율(IFRS연결),4
6,7,[공통]총포괄이익증가율(IFRS연결),3
7,8,[공통]매출액정상영업이익률(IFRS연결),3
8,49,[공통]매출액순이익률(IFRS연결),3
9,41,[공통]총자본사업이익률(IFRS연결),2


In [36]:
corr = pd.concat([variable,var], axis = 1)
corr = corr[['var_index','var','number']]
corr.head(10)

,var_index,var,number
0,24,거래소코드,4
1,14,[공통]총자본증가율(IFRS연결),4
2,47,[공통]자기자본증가율(IFRS연결),4
3,46,[공통]매출액증가율(IFRS연결),4
4,45,[공통]정상영업이익증가율(IFRS연결),4
5,44,[공통]순이익증가율(IFRS연결),4
6,7,[공통]총포괄이익증가율(IFRS연결),3
7,8,[공통]매출액정상영업이익률(IFRS연결),3
8,49,[공통]매출액순이익률(IFRS연결),3
9,41,[공통]총자본사업이익률(IFRS연결),2


In [50]:
corr.loc[:5,'var']

0                    거래소코드
1       [공통]총자본증가율(IFRS연결)
2      [공통]자기자본증가율(IFRS연결)
3       [공통]매출액증가율(IFRS연결)
4    [공통]정상영업이익증가율(IFRS연결)
5       [공통]순이익증가율(IFRS연결)
Name: var, dtype: object

In [78]:
nc_deep = nc[corr.loc[1:5,'var']]

nc_deep['종가'] = nc['종가']

In [91]:
len(x_data.columns)

5

In [80]:
nc_deep = nc_deep[['[공통]총자본증가율(IFRS연결)', '[공통]자기자본증가율(IFRS연결)', '[공통]매출액증가율(IFRS연결)', '[공통]정상영업이익증가율(IFRS연결)', '[공통]순이익증가율(IFRS연결)', '종가']]

In [81]:
nc_deep.columns[-1]

'종가'

In [94]:
import tensorflow as tf
from tqdm import tqdm_notebook
tf.set_random_seed(777)  # for reproducibility

x_data = nc_deep.loc[:,nc_deep.columns[:-1]]
y_data = nc_deep.loc[:,[nc_deep.columns[-1]]] # 괄호를 한번 더 씌어야 한다는 거 유의!!!!!!


# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 5])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([5, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [101]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in tqdm_notebook(range(2000001)):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})

    if step % 100 == 0 or step < 10 :
        print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, hy_val))


In [100]:
nc_deep['종가']

0     150500
1     162000
2     248500
3     182500
4     182000
5     198500
6     213000
7     236000
8     247500
9     378500
10    447500
11    371000
Name: 종가, dtype: int64